In [1]:
import numpy as np


In [2]:
import sympy as sp

In [22]:
def golden_search(f,a,b,tol=1e-2):
    #tol = length of interval of uncertainity
    r = (np.sqrt(5) - 1) / 2
    i =0
    while abs(b - a) >tol:
        x1 = a + (1-r)*(b - a)
        x2 = a + r*(b - a)
        if f(x1) < f(x2):
            b = x2
        else:
            a = x1
        i+= 1
        print(f'iter:{i} | interval:[{a},{b}]')
    x_min = (a+b)/2
    return x_min, f(x_min), i

In [20]:
def f(x):
    return x**2

In [21]:
x_min, f_min, iters = golden_search(f, a=-5, b=15)
print(f"Minimum at x = {x_min}, f(x) = {f_min}, iterations = {iters}")

iter:1 | interval:[-5,7.360679774997898]
iter:2 | interval:[-5,2.6393202250021046]
iter:3 | interval:[-2.0820393249936906,2.6393202250021046]
iter:4 | interval:[-2.0820393249936906,0.8359213500126197]
iter:5 | interval:[-0.9674775249768661,0.8359213500126197]
iter:6 | interval:[-0.27864045000420534,0.8359213500126197]
iter:7 | interval:[-0.27864045000420534,0.41019662496845566]
iter:8 | interval:[-0.27864045000420534,0.14708427503995875]
iter:9 | interval:[-0.11602807488853825,0.14708427503995875]
iter:10 | interval:[-0.11602807488853825,0.046584300227128866]
iter:11 | interval:[-0.05391567458570103,0.046584300227128866]
iter:12 | interval:[-0.015528100075708366,0.046584300227128866]
iter:13 | interval:[-0.015528100075708366,0.022859474434284308]
iter:14 | interval:[-0.015528100075708366,0.0081967257171362]
iter:15 | interval:[-0.006466023000011915,0.0081967257171362]
iter:16 | interval:[-0.006466023000011915,0.002596054075684539]
Minimum at x = -0.0019349844621636882, f(x) = 3.7441648

In [35]:
def fib(n): 
    fn = [0, 1,]
    for i in range(2, n+1):
        fn.append(fn[i-1] + fn[i-2])
    return fn

In [63]:
def fib_search(f,a,b,n):
    F = fib(n)
    L0 = b - a
    #print(f"iter:{1} | interval: [{a},{b}]")
    for i in range(2,n+1):
        L0i = (F[n-i]/F[n])*L0
        x1 = a + L0i
        x2 = b - L0i
        if f(x1)>f(x2):
            a = x1
        else:
            b = x2
        print(f"iter:{i-1} | interval: [{a},{b}]")
    x_min = (a+b)/2
    return x_min, f(x_min), n

In [67]:
fib_search(f,-5,15,8)

iter:1 | interval: [-5,7.380952380952381]
iter:2 | interval: [-5,2.6190476190476195]
iter:3 | interval: [-2.1428571428571432,2.6190476190476195]
iter:4 | interval: [-2.1428571428571432,0.7142857142857149]
iter:5 | interval: [-1.190476190476191,0.7142857142857149]
iter:6 | interval: [-0.23809523809523858,0.7142857142857149]
iter:7 | interval: [-0.23809523809523858,0.7142857142857149]


(0.23809523809523814, 0.056689342403628135, 8)

In [101]:
def newton(f,x_vals,tol=1e-3):
    n = len(x_vals)
    x_syms = sp.symbols(f"x1:{n+1}")

    grad_f = sp.Matrix([sp.diff(f,var) for var in x_syms])
    hess_f = sp.Matrix(n, n, lambda i, j: sp.diff(f, x_syms[i], x_syms[j]))

    grad_func = sp.lambdify(x_syms, grad_f, "numpy")
    hess_func = sp.lambdify(x_syms, hess_f, "numpy")
    func = sp.lambdify(x_syms, f, "numpy")

    i=0

    while True:
        grad = np.array(grad_func(*x_vals), dtype=float).flatten()
        if np.linalg.norm(grad) <= tol:
            print("Gradient tends to 0")
            break
        hess = np.array(hess_func(*x_vals), dtype=float)
        try:
            delta = np.dot(np.linalg.inv(hess),grad)
        except np.linalg.LinAlgError:
            print("Hessian matrix is singular.Stopping")
            break
        x = x_vals - delta
        x_vals = x
        print(f"Iter {i+1}: x = {x_vals}, f(x) = {func(*x_vals):.6f}, ||grad|| = {np.linalg.norm(grad):.6f}")
        i+=1
        
    return x_vals

In [102]:
x1, x2 = sp.symbols('x1 x2 ')
f = x1**2 + 2*x2 + x2**2 + 4

result = newton(f, [2,1])
print("Minimum at:", result)

Iter 1: x = [ 0. -1.], f(x) = 3.000000, ||grad|| = 5.656854
Gradient tends to 0
Minimum at: [ 0. -1.]


In [9]:
def line_search(f, grad, x, delta, c1=1e-4, rho=0.5):
    
    alpha = 1.0
    fx = f(*x)
    while f(*(x + alpha * delta)) > fx + c1 * alpha * np.dot(grad, delta):
        alpha *= rho
    return alpha

In [124]:
def steepest_descent(f,x_vals,tol = 1e-5):
    n = len(x_vals)
    x_syms = sp.symbols(f"x1:{n+1}")

    grad_f = sp.Matrix([sp.diff(f,var) for var in x_syms])

    grad_func = sp.lambdify(x_syms, grad_f, "numpy")
    func =  sp.lambdify(x_syms, f, "numpy")

    i = 0
    while True:
        grad = np.array(grad_func(*x_vals), dtype=float).flatten()
        if np.linalg.norm(grad) <= tol:
            print("Gradient tends to 0")
            break

        d = -grad
        alpha = line_search(func, grad, x_vals, d)
        x_vals = x_vals + alpha * d

        print(f"Iter {i+1}: alpha:{alpha}, x = {x_vals}, f(x) = {func(*x_vals):.6f}, ||grad|| = {np.linalg.norm(grad):.6f}")

    return x_vals

In [125]:
x1, x2 = sp.symbols('x1 x2 ')
f = x1**2 + 2*x2 + x2**2 + 4

result = steepest_descent(f, [2,1])
print("Minimum at:", result)

Iter 1: alpha:0.5, x = [ 0. -1.], f(x) = 3.000000, ||grad|| = 5.656854
Gradient tends to 0
Minimum at: [ 0. -1.]


In [142]:
def Q_conjugate_no_direction(f, x_vals, Q, tol = 1e-2):
    n = len(x_vals)
    x_syms = sp.symbols(f"x1:{n+1}")

    grad_f = sp.Matrix([sp.diff(f,var) for var in x_syms])

    grad_func = sp.lambdify(x_syms, grad_f, "numpy")
    func =  sp.lambdify(x_syms, f, "numpy")

    i = 0
    grad = np.array(grad_func(*x_vals),dtype = float).flatten()
    d = -grad
    while True:
        
        Qd = Q @ d
        alpha = -(grad.T @ d)/(d.T @ Qd)
        x_vals = x_vals + alpha*d
        grad = np.array(grad_func(*x_vals),dtype = float).flatten()
        print(f"Iter {i+1}: alpha = {alpha:.6f}, x = {x_vals}, f(x) = {func(*x_vals):.6f}, ||grad|| = {np.linalg.norm(grad):.6f}, direction : {d}")
        if np.linalg.norm(grad) <= tol:
            print("Gradient tends to 0")
            break
        
        beta = -(grad.T @ Qd)/(d.T @ Qd)
        d = -grad + beta*d
        i += 1
    return x_vals

In [143]:
x1, x2 = sp.symbols('x1 x2 ')
f = x1 - x2 + 2*x1**2 + 2*x1*x2 + x2**2
x0 = np.array([0.0, 0.0])
Q = np.array([[4, 2], [2, 2]])
result = Q_conjugate_no_direction(f, x0, Q)
print("Minimum at:", result)

Iter 1: alpha = 1.000000, x = [-1.  1.], f(x) = -1.000000, ||grad|| = 1.414214, direction : [-1.  1.]
Iter 2: alpha = 0.125000, x = [-0.75  1.  ], f(x) = -1.125000, ||grad|| = 0.500000, direction : [2. 0.]
Iter 3: alpha = 0.200000, x = [-0.7  1.1], f(x) = -1.150000, ||grad|| = 0.447214, direction : [0.25 0.5 ]
Iter 4: alpha = 0.200000, x = [-0.8  1.1], f(x) = -1.170000, ||grad|| = 0.400000, direction : [-5.00000000e-01 -7.21644966e-16]
Iter 5: alpha = 0.200000, x = [-0.76  1.18], f(x) = -1.186000, ||grad|| = 0.357771, direction : [0.2 0.4]
Iter 6: alpha = 0.200000, x = [-0.84  1.18], f(x) = -1.198800, ||grad|| = 0.320000, direction : [-4.00000000e-01 -1.55431223e-15]
Iter 7: alpha = 0.200000, x = [-0.808  1.244], f(x) = -1.209040, ||grad|| = 0.286217, direction : [0.16 0.32]
Iter 8: alpha = 0.200000, x = [-0.872  1.244], f(x) = -1.217232, ||grad|| = 0.256000, direction : [-3.20000000e-01 -4.74620343e-15]
Iter 9: alpha = 0.200000, x = [-0.8464  1.2952], f(x) = -1.223786, ||grad|| = 0.22

Iter 1: alpha = 1.000000, x = [-1.  1.], ||grad|| = 1.414214, direction = [0. 2.]
Gradient tends to 0
Minimum at: [-1.   1.5]


In [70]:
Q = np.array([[2, -3], [-3, 5]])  
b = np.array([0, -1])  
def quadratic_function(x):  
    return 0.5 * np.dot(x, np.dot(Q, x)) - np.dot(b, x)

# Gradient of quadratic function
def gradient(x):
    return np.dot(Q, x) - b

In [71]:
x0 = np.array([0.0, 0.0])
quadratic_function(x0)

[[ 2 -3]
 [-3  5]]


np.float64(0.0)

In [72]:
np.linalg.norm(gradient(x0))

np.float64(1.0)

In [7]:
def q_con_dir(f,x_vals,Q,d,tol=1e-3):
    n = len(x_vals)
    x_syms = sp.symbols(f"x1:{n+1}")

    grad_f = sp.Matrix([sp.diff(f,var) for var in x_syms])

    grad_func = sp.lambdify(x_syms,grad_f,'numpy')
    func = sp.lambdify(x_syms,f,'numpy')

    for i in range(d.shape[0]):
        grad = np.array(grad_func(*x_vals),dtype = float).flatten()
        if np.linalg.norm(grad) <= tol:
            break
        Qd = np.dot(Q,d[i])
        alpha = -np.dot(grad,d[i])/np.dot(d[i],Qd)
        x_vals = x_vals + alpha*d[i]
        print(f"Iter:{i+1}: alpha:{alpha}, x:{x_vals}, f(x):{func(*x_vals)},direction: {d[i]}, grad:{np.linalg.norm(grad)}")
    return x_vals

In [8]:
x1, x2 = sp.symbols('x1 x2 ')
f = x1**2 + 2*x2**2 + x1 - x2 + 1
x0 = np.array([0.0, 0.0])
Q = np.array([[2, 0], [0, 4]])
d = np.array([[1,0],[0,1]])
q_con_dir(f,x0,Q,d,tol=1e-3)

Iter:1: alpha:-0.5, x:[-0.5  0. ], f(x):0.75,direction: [1 0], grad:1.4142135623730951
Iter:2: alpha:0.25, x:[-0.5   0.25], f(x):0.625,direction: [0 1], grad:1.0


array([-0.5 ,  0.25])

In [32]:
def bfgs(f,x_vals,tol=1e-3):
    n = len(x_vals)
    x_syms = sp.symbols(f"x1:{n+1}")

    grad_f = sp.Matrix([sp.diff(f,var) for var in x_syms])

    grad_func = sp.lambdify(x_syms,grad_f,'numpy')
    func = sp.lambdify(x_syms,f,'numpy')

    B = np.eye(n)
    H = np.linalg.inv(B)

    i=0
    while True:
        grad = np.array(grad_func(*x_vals),dtype = float).flatten()
        if np.linalg.norm(grad) <=tol:
            break
        d = -(H @ grad)
        alpha = line_search(func,grad,x_vals,d)
        x_new = x_vals + alpha * d
        grad_new = np.array(grad_func(*x_new), dtype=float).flatten()

        s = x_new - x_vals
        y = grad_new - grad
        
        B =  B - ((B @ s) @ (s.T @ B))/(s.T @ (B @ s)) + (y @ y.T)/(y.T @ s)
        try: 
            H = np.linalg.inv(B)
        except np.linalg.LinAlgError:
            print("Hessian Matrix is Singular")
            B = np.eye(n)
            H = np.linalg.inv(B)
        print(f"Iter:{i+1}: alpha:{alpha}, x:{x_vals}, f(x):{func(*x_vals)}, grad:{np.linalg.norm(grad)}")
        x_vals = x_new
        i+=1
    return x_vals

In [34]:
x1, x2 = sp.symbols('x1 x2 ')
f = x1 - x2 + 2*x1**2 + 2*x1*x2 + x2**2
x0 = np.array([0.0, 0.0])

result = bfgs(f, x0)
print("Minimum at:", result)

Iter:1: alpha:1.0, x:[0. 0.], f(x):0.0, grad:1.4142135623730951
Iter:2: alpha:1.0, x:[-1.  1.], f(x):-1.0, grad:1.4142135623730951
Iter:3: alpha:1.0, x:[-0.66666667  1.33333333], f(x):-1.1111111111111112, grad:1.0540925533894592
Iter:4: alpha:1.0, x:[-1.09009009  1.57657658], f(x):-1.2417011606200794, grad:0.20896240549086864
Iter:5: alpha:1.0, x:[-0.98606473  1.50042176], f(x):-1.2495996836948509, grad:0.06345326376434153
Iter:6: alpha:1.0, x:[-1.00418426  1.51017277], f(x):-1.2499466297118824, grad:0.012508801483888572
Iter:7: alpha:1.0, x:[-1.00109193  1.50489657], f(x):-1.249984332405311, grad:0.009345380252848748
Iter:8: alpha:1.0, x:[-1.00242822  1.50137642], f(x):-1.2499929974672765, grad:0.007270988881049321
Iter:9: alpha:1.0, x:[-0.99938063  1.49956757], f(x):-1.2499995814253768, grad:0.0016554238906691695
Minimum at: [-1.0001155   1.50007145]
